**Import modules et données puis préparation**

In [1]:
import math
from random import randrange
import numpy as np
import pandas as pd
df = pd.read_csv("info_communes_parc.csv", sep=";")

In [2]:
elite = 0.2
mutations = 0.1
taille_echantillon = 5000
scores = pd.DataFrame({"kms":[],"pois":[],"hotel_j3":[],"hotel_j7":[]})
nb_etapes = 10

In [3]:
def distance (depart, arrivee) :
    tmp = df.loc[df["insee"].isin([depart,arrivee]), "lat_centre":"long_centre"]
    x_dep = tmp.iloc[0,1]
    y_dep = tmp.iloc[0,0]
    x_arr = tmp.iloc[1,1]
    y_arr = tmp.iloc[1,0]
    distance = math.acos(math.sin(math.radians(x_dep))*math.sin(math.radians(x_arr))+math.cos(math.radians(x_dep))*math.cos(math.radians(x_arr))*math.cos(math.radians(y_dep-y_arr)))*6371
    return round(distance,1)
noms_villes = df["insee"].tolist()

In [4]:
def generateur_aleatoire (n_etapes, n_iterations, liste) :
    tmp_array = np.zeros(shape=(n_iterations,n_etapes))
    for h in range(n_iterations) :
        liste_voyage = []
        tmp_liste = liste.copy()
        for i in range(n_etapes) :
            nombre_alea = randrange(0, len(tmp_liste), 1)
            liste_voyage.append(tmp_liste[nombre_alea])
            tmp_liste.pop(nombre_alea)
        tmp_array[h] =  liste_voyage
        print("ligne " + str(h),end='\r')
    return tmp_array

In [5]:
generation = generateur_aleatoire(nb_etapes,taille_echantillon, noms_villes)

**Préparation des fonctions et paramètres pour les nouvelles générations**

In [6]:
def notation (matrice) :
    #pense bete tri colonnes array[array[:, 1].argsort()]
    global scores
    #decompte distance
    for i in range(taille_echantillon) :
        distance_totale = 0
        nb_poi = 0
        print("ligne "+ str(i),end='\r')
        for j in range(9) :
            distance_totale += distance(matrice[i,j],matrice[i,j+1])
        add_kms = distance_totale
    #decompte POI
        for k in range(10) :
            valeur = int(df.loc[df["insee"] == matrice[i,k], "nb_poi"])
            nb_poi += valeur
        add_pois = nb_poi
        add_hj3 = df.loc[df["insee"] == matrice[i,2], "nb_hotels_3_etoiles_min"].iloc[0]
        add_j7 = df.loc[df["insee"] == matrice[i,6], "nb_hotels_3_etoiles_min"].iloc[0]
        scores = pd.concat([scores,pd.DataFrame({"kms":[add_kms],"pois":[add_pois],"hotel_j3":[add_hj3],"hotel_j7":[add_j7]})], axis=0)

In [7]:
notation(generation)

/var/folders/lt/rhw2z5td2p1b37fff0l4xc4m0000gn/T/ipykernel_1768/2790168891.py:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  valeur = int(df.loc[df["insee"] == matrice[i,k], "nb_poi"])


In [14]:
scores = scores.sort_values(by="kms",ascending=True)
scores["decile_km"] = 0
for i in range(scores.shape[0]) :
    scores.iloc[i,4] = math.floor(i / (taille_echantillon / 10)) + 1
scores = scores.sort_values(by="pois",ascending=False)
scores["decile_poi"] = 0
for i in range(scores.shape[0]) :
    scores.iloc[i,5] = math.floor(i / (taille_echantillon / 10)) + 1
scores["decile_cumul"] = 10
for i in range(scores.shape[0]) :
    for j in range(1,10) :
        if scores.iloc[i,4] == j and scores.iloc[i,5] == j :
            scores.iloc[i,6] = j
scores = scores.sort_values(by=["decile_cumul","pois","kms"],ascending=[True,False,True])

In [27]:
scores.to_csv("export.csv",sep=";")

In [26]:

scores

,kms,pois,hotel_j3,hotel_j7,decile_km,decile_poi,decile_cumul
0,269.4,78.0,6.0,0.0,1,1,1
0,262.3,73.0,0.0,0.0,1,1,1
0,239.4,70.0,0.0,0.0,1,1,1
0,254.7,69.0,2.0,2.0,1,1,1
0,261.0,68.0,0.0,0.0,1,1,1
...,...,...,...,...,...,...,...
0,318.2,10.0,3.0,0.0,4,10,10
0,321.3,9.0,0.0,0.0,4,10,10
0,326.8,9.0,3.0,0.0,4,10,10
0,303.5,8.0,0.0,0.0,3,10,10
